In [1]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [2]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(5000, replace=False)
#df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
40010,DOes anyone know where or how i can get a copy...,pos
43122,"this is my first review on IMDb, i didn't real...",pos
13189,"I have seen this movie, just once, and I'm loo...",pos
3566,"Please, If you're thinking about renting this ...",neg
44757,I don't know...Maybe it's just because it's an...,pos
...,...,...
6299,The odd mixture of comedy and horror sometimes...,neg
6834,Inspired by a true story tale is full of 1970'...,neg
31326,Dr. K(David H Hickey)has been trying to master...,neg
12385,There have been some great television movies i...,neg


### Palabras de parada

In [3]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...', '..'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '..',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 '

### Normalizar el texto

In [4]:
# LimpiadorTexto es una clase que nosotros creamos.
# Utilizamos la funcion normalizar aquí, en el notebook. y también en la API.
# Por tanto, para no tener código repetido, abstraímos la lógica a una clase externa.
from LimpiadorTexto import LimpiadorTexto
limpiador = LimpiadorTexto(palabras_de_parada_ingles)
df_criticas = df_criticas.apply(limpiador.normalizar_fila, axis=1)
df_criticas

,Review,Label
40010,anyone know get copy film searching way long s...,pos
43122,first review imdb really want write one since ...,pos
13189,seen movie looking forward see dear david beli...,pos
3566,please thinking renting movie thinking watchin...,neg
44757,know maybe impressive tribute muslim religious...,pos
...,...,...
6299,odd mixture comedy horror sometimes works some...,neg
6834,inspired true story tale full 1970 feeling dis...,neg
31326,dr k david h hickey trying master formula woul...,neg
12385,great television movies past epics roots lones...,neg


### Obtener el vocabulario del problema

In [5]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [6]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas))
len(vocabulario_problema)

49519

### One Hot Encoding

In [7]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [8]:
one_hots = []
indices = []
for indice, fila in df_criticas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [9]:
from sklearn.model_selection import train_test_split
# se cambia el nombre de la variable 'one_hots' para mas claridad
X = one_hots
Y = df_criticas['Label'].ravel()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

### Liberar la memoria RAM

In [11]:
import gc
lst = [df_criticas]
del df_criticas, one_hots, X, Y
del lst
gc.collect()

140

## Entrenar diferentes modelos con diferentes parámetros para encontrar el que produzca mejores resultados

Se entrenarán diferentes modelos con diferentes parámetros. Se guardará las métricas de cada modelo en un archivo para que luego puedan ser comparadas, y así determinar el mejor clasificador.

In [3]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

def guardar_objeto(nombre_objeto, objeto):
    ruta_archivo = os.path.join(nombre_objeto)
    archivo = open(ruta_archivo, 'wb')
    pickle.dump(objeto, archivo)
    archivo.close()
    
def obtener_metricas(clasificador):
    predicciones = clasificador.predict(X_test)
    metricas = {
        'accuracy': accuracy_score(Y_test, predicciones),
        'ṕrecision': precision_score(Y_test, predicciones, pos_label='pos'),
        'recall': recall_score(Y_test, predicciones, pos_label='pos'),
        'f1': f1_score(Y_test, predicciones, pos_label='pos')
    }

### Regresión logística con los parámetros:

- random_state=42
- solver='sag'
- max_iter=300

In [14]:
from sklearn.linear_model import LogisticRegression
import pickle

clasificador_reg_log = LogisticRegression(random_state=42, solver='sag', max_iter=300, n_jobs=-1)
clasificador_reg_log.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador_reg_log)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'sag'
metricas['random_state'] = 42
metricas['max_iter'] = 300
guardar_objeto('metricas-reg-log1.pkl', metricas)

/home/jordanmontt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

### Regresión logística con los parámetros:

- random_state=42
- solver='saga'
- n_jobs=-1

In [ ]:
clasificador_reg_log = LogisticRegression(random_state=42, solver='saga', max_iter=300, n_jobs=-1)
clasificador_reg_log.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador_reg_log)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'saga'
metricas['random_state'] = 42
metricas['max_iter'] = 300
guardar_objeto('metricas-reg-log2.pkl', metricas)

### Regresión logística con los parámetros:

- random_state=42
- solver='liblinear'
- n_jobs=-1

In [ ]:
clasificador_reg_log = LogisticRegression(random_state=42, solver='liblinear', n_jobs=-1)
clasificador_reg_log.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador_reg_log)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'liblinear'
metricas['random_state'] = 42
guardar_objeto('metricas-reg-log3.pkl', metricas)

### Regresión logística con los parámetros:

- solver='lbfgs'
- n_jobs=-1

In [ ]:
clasificador_reg_log = LogisticRegression(solver='lbfgs', n_jobs=-1)
clasificador_reg_log.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador_reg_log)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'lbfgs'
guardar_objeto('metricas-reg-log4.pkl', metricas)

### Regresión logística con los parámetros:

- solver='lbfgs'
- n_jobs=-1

In [ ]:
clasificador_reg_log = LogisticRegression(solver='newton-cg', n_jobs=-1)
clasificador_reg_log.fit(X_train, Y_train)
metricas = obtener_metricas(clasificador_reg_log)
metricas['model'] = 'LogisticRegression'
metricas['solver'] = 'newton-cg'
guardar_objeto('metricas-reg-log5.pkl', metricas)

### Guardar el clasificador como un archivo

In [14]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [15]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [16]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()